# Problem 154

In this notebook, I want to collect and organize the knowledge I’ve gained so far throughout this challenge. I’ll use Problem 154 as a way to tell the story of this learning journey and a resume of all the techniques that I found.

## Credits and Acknowledgments

I’d like to thank all the participants — especially @jazivxt, @tonylica, @garrymoss, and @kenkrige — for teaching me so many valuable things along the way.


In [ ]:
task_num = 154
import sys
sys.path.append("/kaggle/input/google-code-golf-2025/code_golf_utils")
from code_golf_utils import *
show_legend()

In [ ]:
examples = load_examples(task_num)
show_examples(examples['train'] + examples['test'])

### First version explanation
There is a recursive call that runs the function twice. First of all there is the transposition of the matrix `g`
```python
 *g,=map(list,zip(*g))
```
The expression `zip(*g)` takes rows and turns them into colums, so the `zip` is *excatly* the transpose operator, and `map(list, ...)` ensures that each row is converted back into a list 

---

```python
 if max(r.count(2)for r in g)<5:
```

This condition checks how many times the number `2` appears in each row. I use this statement to check if I have to make the transformation or not.

---

```python
 b,*_,d=(j for r in g for j in R if r[j]==2)
```
  
  This line finds all the **column indices `j`** in the grid where the value is `2`.
  It then unpacks the **first** and **last** indices into `b` (begin) and `d` (end).
  The `*_,` part ignores all the middle values.
  So effectively, `b` and `d` represent the **leftmost and rightmost columns** where a `2` is found.

---

```python
  for r in g:
   for j in R:
    if r[j]>2and(d<j or j<b):r[j]=0;r[2*(b,d)[j>=b+d+1>>1]-j]=5
```

* This double loop goes through every cell of the grid.
* The condition `r[j] > 2` means it only affects cells whose value is greater than `2`, i.e. `5` in this example.
* The subcondition `(d < j or j < b)` ensures it only modifies cells outside the range between `b` and `d`.
* When that happens:

  * The current cell `r[j]` is set to `0`.
  * Then, it computes a mirrored position around the midpoint between `b` and `d` and sets that cell to `5`.

---

```python
 return k*g or p(g, 1)
```

* If `k` is `0`, the expression `k*g` evaluates to `0` (which is falsy), so the function recursively calls itself: `p(g, 1)`.
* On the recursive call, `k` becomes `1`, so `k * g` evaluates to the grid itself, and the recursion stops.

In [ ]:
%%writefile task.py
def p(g,k=0,R=range(15)):
 *g,=map(list,zip(*g))
 if max(r.count(2)for r in g)<5:
  b,*_,d=(j for r in g for j in R if r[j]==2)
  for r in g:
   for j in R:
    if r[j]>2and(d<j or j<b):r[j]=0;r[2*(b,d)[j>=b+d+1>>1]-j]=5
 return k*g or p(g,1)

In [ ]:
verify_program(task_num, examples)

### Compression
After that I see really good notebook of @garrymoss and I use his well explained tricks of variables renaming and code compression to go from `243` bytes to around `220` bytes.

### Regex
After the good notebooks of @kenkrige I decide to starts with some regex and here there is the code with I come.

I cycle for time with `for _ in g*4` and I rotate the matrix at every iteration with `zip(*g[::-1])`. Here I use a regex, first I transform each row in a nice string with `str(r)[1::3]` apply the regex and transform back to a list with `[*map(int, <str> )]`.

The regex mirrors the `5` cells respect the red one only on the left of the leftmost red cell.
 The pattern `'(.*[^0].*)020(.*)020(.*)'` looks for a specific structure:

   * `(.*[^0].*)` — a sequence that contains at least one non-zero character (this becomes `m[1]`).
   * `020` — a literal marker separating sections.
   * `(.*)` — another group of arbitrary characters (`m[2]`).
   * Another `020` separator.
   * `(.*)` — the remaining part of the string (`m[3]`).

  The lambda function defines how to build the replacement string:

   ```python
   lambda m: '0'*(l := len(m[1])) + '020' + m[1][::-1] + m[2][l:] + '020' + m[3]
   ```

   * I assign `l` to the length of the first captured group (`m[1]`).
   * `'0' * l` creates a sequence of zeros equal in length to `m[1]`.
   * `m[1][::-1]` reverses the first group.
   * `m[2][l:]` removes the first `l` characters from the second group.
   * I then concatenate these components with the literal markers `'020'` to reconstruct a new pattern:

     ```
     [zeros] + '020' + [reversed first part] + [trimmed middle] + '020' + [last part]
     ```

The regex with `re.sub` permits to reduce a lot the length of the program, mantaining almost the same logic.


In [ ]:
%%writefile task.py
import re
p=lambda g:[(g:=[[*map(int,re.sub('(.*[^0].*)020(.*)020(.*)',lambda m:'0'*(l:=len(m[1]))+'020'+m[1][::-1]+m[2][l:]+'020'+m[3],str(r)[1::3]))]for r in zip(*g[::-1])])for _ in g*4]and g

In [ ]:
verify_program(task_num, examples)

### Community improvements
Thanks to @tonylica that squeeze out my notebook improving the regex expression (-3 Bytes) and the final result extraction (-2 Byte). Also I notice that the parenthesis around the walrus are not needed (-2 Byte). Bringing the solution to `187` Bytes.

In [ ]:
%%writefile task.py
import re;p=lambda g:[g:=[[*map(int,re.sub(r'(.*5.*)(020)(.*)\2(.*)',lambda m:'0'*len(x:=m[1])+m[2]+x[::-1]+m[3][len(x):]+m[2]+m[4],str(r)[1::3]))]for r in zip(*g[::-1])]for _ in g*4][3]

In [ ]:
verify_program(task_num, examples)

### Conclusion

The regular expression allows me to go even beyond 200 bytes.

---

This notebook summarizes all the tricks and techniques I have learned so far. I hope you enjoy it and can learn from my experience.

---

This represents the best code I have produced so far, though I am still about 100 bytes above the optimal solution. I am fascinated by how much further I might go with new tricks or a clever approach!